<a href="https://colab.research.google.com/github/arnottrcaiado/pet-centrocar-2021/blob/main/pdf_clientes_slice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto PET - Centrocar
# Tratamento de dados de Clientes - Conversao de PDF para CSV
# usando recursos de Slice

In [45]:
from google.colab import files
uploaded = files.upload()

Saving grupos (1).csv to grupos (1).csv


In [3]:
!ls -la

total 1380
drwxr-xr-x 1 root root    4096 Dec  2 10:31 .
drwxr-xr-x 1 root root    4096 Dec  2 10:30 ..
-rw-r--r-- 1 root root 1394877 Dec  2 10:32 CLIENTES.PDF
drwxr-xr-x 4 root root    4096 Nov 18 14:35 .config
drwxr-xr-x 1 root root    4096 Nov 18 14:36 sample_data


In [4]:
import pandas as pd

In [5]:
import io

In [ ]:
!ls -la

total 5404
drwxr-xr-x 1 root root    4096 Nov 29 14:37  .
drwxr-xr-x 1 root root    4096 Nov 29 12:10  ..
-rw-r--r-- 1 root root 1394877 Nov 29 12:19 'CLIENTES (1).PDF'
-rw-r--r-- 1 root root 1815844 Nov 29 14:36  Clientes_Codigo.csv
drwxr-xr-x 4 root root    4096 Nov 18 14:35  .config
-rw-r--r-- 1 root root 2300398 Nov 29 14:37 'grupos (1).csv'
drwxr-xr-x 1 root root    4096 Nov 18 14:36  sample_data


In [ ]:
!pip3 install fitz

In [ ]:
!pip install pymupdf

In [10]:
# importa biblioteca
import fitz

# faz a leitura
with fitz.open( 'CLIENTES.PDF' ) as pdf:
    texto = ""
    for pagina in pdf:
        texto += pagina.getText()
texto

Deprecation: 'getText' removed from class 'Page' after v1.19.0 - use 'get_text'.


"===============================================================================================================================================\n0001 - CENTROCAR                                                                                                               Data: 25/11/2021\nPCCDCLI0                                                         Persona-arnott                                                Nro:00487JAN.REL\nCADASTRO                                                                                                                       Pagina....: 0001\n===============================================================================================================================================\nNome/razao Social                                            Código  Data Nasci Bairro               Cidade\n-----------------------------------------------------------------------------------------------------------------------------------------------\nCLEIDENILSON JOSE DE

In [11]:
linhas = texto.splitlines()

In [38]:
colunas =['Nome','Codigo','Data Nasc','Bairro','Cidade']
dados =[]

In [39]:
def trataLinhaSlice( linha ):
  nome = linha[0:50].rstrip()
  codigo = linha[61:68]
  data = linha[70:79]
  bairro = linha[80:101].rstrip()
  cidade = linha[101:len(linha)]
  return nome, codigo, data, bairro, cidade

In [40]:
for i in linhas[7:-1] :
  dados.append( trataLinhaSlice(i))

In [41]:
df = pd.DataFrame(dados, columns = colunas)


In [43]:
df.to_csv( 'Clientes_Codigo.csv',sep=';',index=False)

In [46]:
dfn=pd.read_csv('grupos (1).csv')

In [47]:
!ls -la

total 5680
drwxr-xr-x 1 root root    4096 Dec  2 10:42  .
drwxr-xr-x 1 root root    4096 Dec  2 10:30  ..
-rw-r--r-- 1 root root 2099857 Dec  2 10:42  Clientes_Codigo.csv
-rw-r--r-- 1 root root 1394877 Dec  2 10:32  CLIENTES.PDF
drwxr-xr-x 4 root root    4096 Nov 18 14:35  .config
-rw-r--r-- 1 root root 2300398 Dec  2 10:43 'grupos (1).csv'
drwxr-xr-x 1 root root    4096 Nov 18 14:36  sample_data


In [48]:
df['Sexo']=''

In [49]:
import requests, json

In [ ]:
for i in range( len(df )):
  # chamar a api que busca sexo a partir de um nome
  resposta = requests.get( 'https://centrocarpet.pythonanywhere.com/buscasexo?nome='+df.loc[i,'Nome'])
  df.loc[i,'Sexo'] = resposta.json()['Sexo']
  # print( df.loc[i,'Consultor']+" Sexo:"+resposta.json()['Sexo'])

In [ ]:
for i in range( len(df )):
  # chamar a api que busca sexo a partir de um nome
  resposta = procNome( df.loc[i,'Nome'])
  df.loc[i,'Sexo'] = resposta
  print( df.loc[i])
  # print( df.loc[i,'Consultor']+" Sexo:"+resposta.json()['Sexo'])

In [51]:
def procNome( nome ):
    if nome != None :
        lnome = nome.split()
        nome = lnome[0]
        nome = nome.upper()

        busca = dfn.loc[dfn['name']== nome]
        if len(busca) > 0 :
            nomeEncontrado = busca['name'].to_list()
            sexEncontrado = busca['classification'].to_list()
            # return json.dumps({"Nome": nomeEncontrado[0], "Sexo" : sexEncontrado[0] })
            return sexEncontrado[0]
        else:
            sexo = procNomes( nome )
            # return json.dumps({"Nome" : nome, "Sexo": sexo })
            return sexo

# -------------------------------------------------------------------------------------------
# funcao para percorrer o dataframe procurando os nomes alterativos
# chamada quando o nome principal, indexado, não é localizado
def procNomes( nome ) :
    for i in range(len(dfn)) :
      nam = dfn['names'][i].split(sep='|')
      for j in range(len(nam)):
        if nome == nam[j] :
            sex = dfn['classification'][i]
            return str(sex)
    return ""


In [53]:
df.to_csv( 'Clientes_CodNomeSexo.csv',sep=';',index=False)

In [54]:
from google.colab import drive
drive.mount ('/drive')

Mounted at /drive


In [55]:
df.to_csv('/drive/MyDrive/Clientes_CodNomeSexo.csv',sep=';',index=False)

In [56]:
df1 = df[['Codigo','Data Nasc','Bairro','Cidade','Sexo']]


In [57]:
df1.to_csv('/drive/MyDrive/Clientes_CodSexo.csv',sep=';',index=False)